In [4]:
!pip install matplotlib

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import cv2


if torch.cuda.is_available():
    DEVICE= torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(torch.__version__,DEVICE)




1.10.0 cuda
